In [ ]:
import wandb

wandb.init(mode="disabled")

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import torch
import os

In [ ]:
# train
yolo = YOLO("yolov8n.pt")
results = yolo.train(data="/kaggle/input/yolo-train/YOLO/yolo.yaml", epochs=10)

In [ ]:
# Export the trained model
model.export(format='torchscript', path='/kaggle/working/yolo_model.pt')


In [ ]:
yolo_trained = YOLO('/kaggle/input/yolo-trained/YOLO_trained_10/best.pt')


In [ ]:
yolo_trained.eval()

In [ ]:
def count_people_from_labels(label_path):
    person_count = 0
    
    # Open the label file and count entries where class_id is 0 (person)
    with open(label_path, 'r') as file:
        for line in file:
            class_id = int(line.split()[0])
            if class_id == 0:  # 0 is the class ID for 'person'
                person_count += 1
    
    return person_count


In [ ]:
from PIL import Image

def count_people_from_yolo(image_path):
    # Load the image
    img = Image.open(image_path)
    
    # Perform inference
    results = model(img)
    
    # Get the detections
    detections = results.pred[0]
    
    # Count the number of people detected
    person_count = 0
    for det in detections:
        class_id = int(det[-1])
        if class_id == 0:  # YOLO class ID for 'person' is 0
            person_count += 1
    
    return person_count


In [ ]:
folder = '/kaggle/input/yolo-train/YOLO/Testing'

actual_counts = []
predicted_counts = []

for filename in os.listdir(folder):
    # Process only .jpg files
    if filename.endswith('.jpg'):
        # Get corresponding image and label paths
        image_path = os.path.join(folder, filename)
        label_path = os.path.join(folder, os.path.splitext(filename)[0] + '.txt')
        
        # Ensure the label file exists
        if os.path.exists(label_path):
            # Count people based on labels (ground truth)
            actual_count = count_people_from_labels(label_path)
            
            # Count people using YOLO model
            predicted_count = count_people_from_yolo(image_path)
            
            # Store the counts
            actual_counts.append(actual_count)
            predicted_counts.append(predicted_count)
            
            print(f"{filename}: Actual = {actual_count}, Predicted = {predicted_count}")


In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

r2 = r2_score(actual_counts, predicted_counts)
mae = mean_absolute_error(actual_counts, predicted_counts)

print(f'R²: {r2}')
print(f'MAE: {mae}')
